In [0]:
%run /Repos/a.torralbo@ucl.ac.uk/ccu013_02/atlas/00_Load_helpers

In [0]:
phe_unroll = spark.sql("""
SELECT 
    d.person_id_deid,
    d.date,
    d.DIAG_4,
    d.source,
    d.source_code,
    r.phecode_unrolled AS phecode
FROM 
  dsa_391419_j3w9t_collab.ccu013_phedatav200 d
LEFT JOIN 
  dsa_391419_j3w9t_collab.ccu013_lkp_phev200_rollmap r
ON 
  d.phecode = r.phecode
AND
 d.DIAG_4 != '-'
 """)

In [0]:
drop_table('ccu013_phedatav200_unrolled')

In [0]:
phe_unroll.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled')

In [0]:
%sql
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled limit 2;

In [0]:
%sql
select count(*) as n_event, count(distinct(person_id_deid)) as n_pat, count(distinct(phecode)) as n_phecode from dsa_391419_j3w9t_collab.ccu013_phedatav200;

### Create v200_unrolled table with first events of cohort before study start plus QC

In [0]:
unrolled_f = spark.sql("""
select 
  p.person_id_deid,  
  p.source, 
  p.phecode, 
  c.sex as person_id_deid_sex,
  min(p.date) as date
from 
  dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled p, 
  dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c
where p.person_id_deid = c.NHS_NUMBER_DEID
group by p.person_id_deid, p.source, p.phecode, c.dob, c.sex
having MIN(p.date) <= to_date('2020-01-23', 'yyyy-MM-dd') and MIN(p.date) >= date(c.dob)
""")

In [0]:
unrolled_f.createOrReplaceTempView('ccu013_phedatav200_unrolled_first')

In [0]:
drop_table("ccu013_phedatav200_unrolled_first")

In [0]:
unrolled_f.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first')

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first limit 3;

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first;

explore sex-dependent restriction in events table - easier for multimorbidity 

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_lkp_phev200_phedict limit 5;

In [0]:
%sql 
select 
  p.person_id_deid, 
  p.source, 
  p.phecode, 
  p.person_id_deid_sex,
  p.date, 
  d.sex as phe_sex, 
  d.phecode
from 
  dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first p, 
  dsa_391419_j3w9t_collab.ccu013_lkp_phev200_phedict d
where (p.phecode = d.phecode) 
and (d.sex = 0  or d.sex = p.person_id_deid_sex);

In [0]:
unrolled_f_sex = spark.sql("""
select 
  p.person_id_deid, 
  p.source,
  p.phecode, 
  p.person_id_deid_sex,
  p.date, 
  d.sex as phe_sex
from 
  dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first p, 
  dsa_391419_j3w9t_collab.ccu013_lkp_phev200_phedict d
where (p.phecode = d.phecode) 
and (d.sex = 0  or d.sex = p.person_id_deid_sex) """)

In [0]:
unrolled_f_sex.createOrReplaceTempView('ccu013_phedatav200_unrolled_first_sex_dep')

In [0]:
drop_table("ccu013_phedatav200_unrolled_first_sex_dep")

In [0]:
unrolled_f_sex.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep')

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep limit 5;

In [0]:
%sql 
select distinct(person_id_deid_sex) from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep
where phe_sex = 1;

In [0]:
%sql 
select distinct(person_id_deid_sex) from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep
where phe_sex = 2;

In [0]:
%sql 
select distinct(person_id_deid_sex) from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep
where phe_sex = 0;

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep;

In [0]:
%sql 
select count(distinct(phecode)) from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep;

In [0]:
%sql 
select count(distinct(phecode)) from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first;